# 1. Download TF Models from Tensorflow Model Zoo

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [ ]:
import os

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:

!git clone https://github.com/tensorflow/models

# 2. Install and Add Protoc to Path 

In [ ]:
# Install Tensorflow Object Detection 
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)
!tar -xf protoc-3.15.6-win64.zip   
os.environ['PATH'] += os.pathsep + os.path.abspath('bin')   
!cd models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd models/research/slim && pip install -e . 

# 3. Verification Script to Tell Our Dependencies is Correct

In [ ]:
# Verify Installation
verified= os.path.join('models','research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {verified}

In [ ]:
import object_detection

# 4. Download the Selected Model

In [ ]:
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'
wget.download(model_url)
pretrained_model = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
!tar -zxvf {pretrained_model+'.tar.gz'}

# 5. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file('ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config')

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile('ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config', "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
label_map_loc = 'TFRecord data format and label map/label_map.pbtxt'
ckpt_loc = 'ssd_mobnet'

In [ ]:
pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join('ssd_mobilenet_v2_320x320_coco17_tpu-8\checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= label_map_loc
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join('TFRecord data format and label map', 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = label_map_loc
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join('TFRecord data format and label map', 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile('ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config', "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [ ]:
pip install --upgrade numpy

In [ ]:
training_script = os.path.join('models', 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
!mkdir {'ssd_mobnet'}

In [ ]:
pipeline_loc = 'ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config'
ckpt_loc = 'ssd_mobnet'

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(training_script, 'ssd_mobnet','ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config')

In [ ]:
print(command)

In [ ]:
!{command}

# 7. Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(training_script, 'ssd_mobnet','ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config', 'ssd_mobnet')

In [ ]:
print(command)

In [ ]:
!{command}

# 8. Load Train Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_loc)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(ckpt_loc, 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Freezing the Graph

In [ ]:
!mkdir {'ssd_mobnet\export'}
ckpt_out = 'ssd_mobnet\export'

In [ ]:
exporter = os.path.join('models', 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(exporter ,pipeline_loc, ckpt_loc, ckpt_out)

In [ ]:
print(command)

In [ ]:
!{command}

# 10. Conversion to TFLite

In [ ]:
!mkdir {'ssd_mobnet\\tflite_out'}
tflite_output = 'ssd_mobnet\\tflite_out'

In [ ]:
tflite_converter = os.path.join('models', 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(tflite_converter ,pipeline_loc, ckpt_loc, tflite_output)

In [ ]:
print(command)

In [ ]:
!{command}

In [ ]:
tflite_fix = os.path.join(tflite_output, 'saved_model')
tflite_model = os.path.join(tflite_output, 'saved_model', 'detect.tflite')

In [ ]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(tflite_fix, tflite_model, )

In [ ]:
print(command)

In [ ]:
!{command}